# What makes Happiness?

For a long time, people have been interested in what makes us happy and how we can improve well-being in society. One big question is whether having more money or wealth actually makes people happier, or if there are other things that matter more. We often hear the saying “money can’t buy happiness,” but the real answer is a bit more complicated than that.

In this data story, we will explore how happiness relates to different economic and social factors using information from the World Happiness Report 2019 and World Development Indicators. This report looks at how happy people are in different countries and compares that with demographic variables like GDP per capita, Gross National Income (GNI), the gini index, unemployment rates, education levels, and life expectancy.

We want to find out if richer countries really have happier people, and if so, how strong this connection is. But we will also look beyond money to see how things like having a job, going to school, and living a long, healthy life affect happiness. For example, being unemployed might make people less happy even if their country is wealthy, and having a good education could improve well-being in ways that money alone can’t.

First, we will compare happiness scores with income indicators like GDP per capita and GNI. Then, we will analyze how unemployment rates relate to happiness. After that, we will look at the gini index, education and life expectancy to see how these factors could affect happiness scores across countries.

By comparing these different aspects, we hope to better understand what really contributes to happiness around the world. This will help us see whether the saying “money buys happiness” really holds. And if it doesn’t hold, we could find out what does make happiness.

In [42]:
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

df = pd.read_csv('final.csv')

df['Happiness_rank'] = df['Happiness'].rank(method='min', ascending=False).astype(int)
total_countries = df['Country'].nunique()

df_gdp  = df.dropna(subset=['GDP_current_usd', 'Happiness', 'Country', 'Happiness_rank'])
df_gni  = df.dropna(subset=['GNI_current_usd', 'Happiness', 'Country', 'Happiness_rank'])
df_gini = df.dropna(subset=['Gini_index',       'Happiness', 'Country', 'Happiness_rank'])

trace_gdp = go.Scatter(
    x=df_gdp['GDP_current_usd'],
    y=df_gdp['Happiness'],
    mode='markers',
    name='GDP (US$)',
    visible=True,
    text=df_gdp['Country'],
    customdata=df_gdp['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "GDP (US$): %{x:$,.0f}<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

trace_gni = go.Scatter(
    x=df_gni['GNI_current_usd'],
    y=df_gni['Happiness'],
    mode='markers',
    name='GNI (US$)',
    visible=False,
    text=df_gni['Country'],
    customdata=df_gni['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "GNI (US$): %{x:$,.0f}<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

trace_gini = go.Scatter(
    x=df_gini['Gini_index'],
    y=df_gini['Happiness'],
    mode='markers',
    name='Gini Index',
    visible=False,
    text=df_gini['Country'],
    customdata=df_gini['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "Gini Index: %{x:.2f}<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

fig = go.Figure(data=[trace_gdp, trace_gni, trace_gini])

steps = [
    dict(
        method='update',
        label='GDP (US$)',
        args=[
            {'visible': [True, False, False]},
            {
                'title': 'Happiness vs GDP (current US$)',
                'xaxis': {'title': 'GDP (current US$)', 'type': 'log'},
                'yaxis': {'title': 'Happiness'}
            }
        ]
    ),
    dict(
        method='update',
        label='GNI (US$)',
        args=[
            {'visible': [False, True, False]},
            {
                'title': 'Happiness vs GNI (current US$)',
                'xaxis': {'title': 'GNI (current US$)', 'type': 'log'},
                'yaxis': {'title': 'Happiness'}
            }
        ]
    ),
    dict(
        method='update',
        label='Gini Index',
        args=[
            {'visible': [False, False, True]},
            {
                'title': 'Happiness vs Gini Index',
                'xaxis': {'title': 'Gini Index', 'type': 'linear'},
                'yaxis': {'title': 'Happiness'}
            }
        ]
    )
]

fig.update_layout(
    sliders=[dict(
        active=0,
        currentvalue={'prefix': 'Metric: '},
        pad={'t': 50},
        steps=steps
    )],
    title="Happiness vs GDP (current US$)",
    width=900,
    height=600,
    margin={'t':80}
)

iplot(fig)


In [43]:
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

# 1) Enable Plotly in the notebook
init_notebook_mode(connected=True)

# 2) Load your data
df = pd.read_csv('final.csv')

# 3) Compute happiness rank (1 = happiest) and total countries
df['Happiness_rank'] = df['Happiness'].rank(method='min', ascending=False).astype(int)
total_countries = df['Country'].nunique()

# 4) Prepare subsets (drop rows missing each metric or rank)
df_edu   = df.dropna(subset=['Edu_expenditure_pct', 'Happiness', 'Country', 'Happiness_rank'])
df_unemp = df.dropna(subset=['Unemployment_rate',    'Happiness', 'Country', 'Happiness_rank'])
df_le    = df.dropna(subset=['Life_expectancy',      'Happiness', 'Country', 'Happiness_rank'])

# 5) Create the three scatter traces
trace_edu = go.Scatter(
    x=df_edu['Edu_expenditure_pct'],
    y=df_edu['Happiness'],
    mode='markers',
    name='Education Expenditure',
    visible=True,
    text=df_edu['Country'],
    customdata=df_edu['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "Education Expenditure: %{x:.1f}% of GDP<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

trace_unemp = go.Scatter(
    x=df_unemp['Unemployment_rate'],
    y=df_unemp['Happiness'],
    mode='markers',
    name='Unemployment Rate',
    visible=False,
    text=df_unemp['Country'],
    customdata=df_unemp['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "Unemployment Rate: %{x:.1f}%<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

trace_le = go.Scatter(
    x=df_le['Life_expectancy'],
    y=df_le['Happiness'],
    mode='markers',
    name='Life Expectancy',
    visible=False,
    text=df_le['Country'],
    customdata=df_le['Happiness_rank'],
    hovertemplate=(
        "Country: %{text}<br>"
        f"Rank: %{{customdata}} out of {total_countries}<br>"
        "Life Expectancy: %{x:.1f} years<br>"
        "Happiness: %{y:.2f}<extra></extra>"
    )
)

# 6) Assemble figure
fig = go.Figure(data=[trace_edu, trace_unemp, trace_le])

# 7) Define slider steps
steps = [
    dict(
        method='update',
        label='Education Expenditure',
        args=[
            {'visible': [True, False, False]},
            {
                'title': 'Happiness vs Education Expenditure',
                'xaxis': {'title': 'Education Expenditure (% of GDP)', 'type': 'linear'},
                'yaxis': {'title': 'Happiness'}
            }
        ]
    ),
    dict(
        method='update',
        label='Unemployment Rate',
        args=[
            {'visible': [False, True, False]},
            {
                'title': 'Happiness vs Unemployment Rate',
                'xaxis': {'title': 'Unemployment Rate (%)', 'type': 'linear'},
                'yaxis': {'title': 'Happiness'}
            }
        ]
    ),
    dict(
        method='update',
        label='Life Expectancy',
        args=[
            {'visible': [False, False, True]},
            {
                'title': 'Happiness vs Life Expectancy',
                'xaxis': {'title': 'Life Expectancy (years)', 'type': 'linear'},
                'yaxis': {'title': 'Happiness'}
            }
        ]
    )
]

# 8) Add the slider to layout
fig.update_layout(
    sliders=[dict(
        active=0,
        currentvalue={'prefix': 'Metric: '},
        pad={'t': 50},
        steps=steps
    )],
    title="Happiness vs Education Expenditure",
    width=900,
    height=600,
    margin={'t':80}
)

# 9) Display inline
iplot(fig)


In [44]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('final.csv')

fig = px.choropleth(
    df,
    locations='Country',
    locationmode='country names',
    color='Happiness',
    hover_name='Country',
    color_continuous_scale='Plasma',
    title='World Happiness Score by Country',
    width=1200,
    height=600
)

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    ),
    margin={"r":0,"t":60,"l":0,"b":0}
)

fig.show()
